In [12]:
!pip install tensorflow==2.0.0

  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d3/9e/a48cd34dd7b672ffc227b566f7d16d63c62c58b542d54efa45848c395dd4/tensorboard-2.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
  

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as data

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, Flatten , LSTM

In [0]:
EPOCHS = 10

batch_size = 64
embedding_size = 64
lstm_nn = 64
h1 = 64

In [1]:
Data, info = data.load('imdb_reviews/subwords32k', with_info=True, as_supervised=True)
#info i.e. features and labels are stored in dictionary
# use as_supervised=True to convert the dictionary to a tuple

NameError: name 'data' is not defined

In [0]:
train , test = Data["train"], Data["test"]

In [0]:
tokenizer = info.features["text"].encoder

In [0]:
train = train.shuffle(1000).padded_batch(batch_size, train.output_shapes)
test = test.padded_batch(batch_size, test.output_shapes)

In [0]:
model = Sequential()
model.add(Embedding(tokenizer.vocab_size,embedding_size))
model.add(Bidirectional(LSTM(lstm_nn)))
model.add(Dense(h1, activation ="relu"))
model.add(Dense(1))

loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [0]:
for epoch in range(EPOCHS):
  accuracy = []
  for batch, (text, label) in enumerate(train.take(-1)):
    with tf.GradientTape() as tape:
      logits = model(text)
      label = tf.expand_dims(label,1)
      loss =  loss_func(label, logits)

      gradients = tape.gradient(loss, model.trainable_variables)
      grads_and_vars = zip(gradients, model.trainable_variables)
      optimizer.apply_gradients(grads_and_vars)
      predictions = tf.cast(tf.math.greater(tf.sigmoid(logits), 0.5), tf.int64)
      accuracy.extend(tf.cast(tf.equal(predictions, label), tf.int64).numpy())

      if batch % 100 == 0:
            print('\n Epoch : {} Batch: {}'.format(epoch, batch))
            print('Loss : {:.3f}'.format(loss.numpy()))
            print('Accuracy: {}'.format(np.mean(accuracy)))
      
            for _, (text, label) in enumerate(test.take(1)):
                logits = model(text)
        
                random_id = np.random.choice(label.shape[0], 5)
                for ix in random_id:
                    print('\n')
                    print(tokenizer.decode(text.numpy()[ix]))
                    print('Label', u'\u2713' if label.numpy()[ix] == 1 else u'\u2715')
                    print('Pred', u'\u2713' if logits.numpy()[ix][0] >= 0.5 else u'\u2715')


 Epoch : 0 Batch: 0
Loss : 0.695
Accuracy: 0.453125


I first saw this film on hbo around 1983 and I loved it! I scoured all of the auction web sites to buy the vhs copy. This is a very good suspense movie with a few twists that make it more interesting. I don't want to say too much else because if you ever get a chance to see it, you'll be glad I didn't say too much!
Label ✓
Pred ✕


This is a good episode, but it's not my favorite. A lot of people love it and on a creative level it's brilliant. Most of the episode has no dialog, which is such a cool idea and "Hush" handles the silence really well. Plus, this episode introduces Tara, who I like a lot. But, I don't like Riely or the Initative or Maggie Walsh and they seem to get a lot of screen time in "Hush." Plus, I don't think the Gentlemen are that scary and I get tired of watching the float around Sunnydale. I know that I'm in the minority, but I tend to fast-forward parts of "Hush." Of course, there are other parts of the episod

In [0]:
test_text_1 = " Mostly the film is an allegory about holding onto a sense of personal worth, despite everything. If the film is perhaps a little slow in its middle passages, maybe that is part of the idea, too, to give us a sense of the leaden passage of time, before the glory of the final redemption."
test_text_1 = tokenizer.encode(test_text_1)

pred = model(tf.convert_to_tensor([test_text_1]))
print(u'\u2713' if pred.numpy()[0, 0] >= 0.5 else u'\u2715')

test_text_2 = "Valentine's Day' is being marketed as a Date Movie. I think it's more of a First-Date Movie. If your date likes it, do not date that person again. And if you like it, there may not be a second date."
test_text_2 = tokenizer.encode(test_text_2)

pred = model(tf.convert_to_tensor([test_text_2]))
print(u'\u2713' if pred.numpy()[0, 0] >= 0.5 else u'\u2715')